#  Data Analysis With LLMs

This was originally based on Datacamp's code-along "Introduction to Large Language Models with GPT & LangChain"

In [ ]:
import os
import openai
from dotenv import load_dotenv
load_dotenv() # Load the .env file

In [ ]:
# langchain useful for simplifying calls, swap LLMs, and more
#!pip install langchain

# Update the typing_extensions package
#%pip install typing_extensions==4.8.0

In [ ]:
# Set openai.api_key to the OPENAI_API_KEY environment variable
openai.api_key = os.environ["OPENAI_KEY"]

In [2]:
# Import the langchain package as lc
import langchain as lc

# From the langchain.chat_models module, import ChatOpenAI
from langchain.chat_models import ChatOpenAI

# From the langchain.schema module, import AIMessage, HumanMessage, SystemMessage
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [3]:
# Import pandas using the alias pd
import pandas as pd

# Import plotly.express using the alias px
import plotly.express as px

# From the IPython.display package, import display and Markdown
from IPython.display import display, Markdown

In [ ]:
# Read the data from electric_cars.csv. Assign to electric_cars.
electric_cars = pd.read_csv("electric_cars.csv")

# Display a description of the numeric columns
print("Description of numeric columns\n")
display(electric_cars.describe())

# Display a description of the text (object) columns
print("Description of text columns\n")
display(electric_cars.describe(include="O"))

# Print the whole dataset
print("The electric cars dataset\n")
electric_cars

### Asking GPT a question

In [5]:
from openai import OpenAI
client = OpenAI(api_key=os.environ['OPENAI_KEY'])

# Define the system message. Assign to system_msg_test.
system_msg_test = """You are a helpful assistant who understands data science.
 You write in a clear language that a ten year old can understand.
 You keep your answers brief."""

# Define the user message. Assign to user_msg_test.
user_msg_test = "Tell me some uses of GPT for data analysis."

# Create a message list from the system and user messages. Assign to msgs_test.
msgs_test = [
    {"role": "system", "content": system_msg_test},
    {"role": "user", "content": user_msg_test}
]

# Send the messages to GPT. Assign to rsps_test.
rsps_test = completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=msgs_test
)

In [ ]:
# Print the whole response
print("The whole response\n")
display(rsps_test.model_dump_json())

print("\n\n----\n\n")

# Print just the response's content
print("Just the response's content\n")
print(rsps_test.choices[0].message.content)

### Task 3: Asking a Question About the Dataset

In [7]:
# A description of the dataset
dataset_description = """
You have a dataset about electric cars registered in Washington state, USA in 2020. It is available as a pandas DataFrame named `electric_cars`.

Each row in the dataset represents the count of the number of cars registered within a city, for a particular model.

The dataset contains the following columns.

- `city` (character): The city in which the registered owner resides.
- `county` (character): The county in which the registered owner resides.
- `model_year` (integer): The [model year](https://en.wikipedia.org/wiki/Model_year#United_States_and_Canada) of the car.
- `make` (character): The manufacturer of the car.
- `model` (character): The model of the car.
- `electric_vehicle_type` (character): Either "Plug-in Hybrid Electric Vehicle (PHEV)" or "Battery Electric Vehicle (BEV)".
- `n_cars` (integer): The count of the number of vehicles registered.
"""

# Create a task for the AI. Assign to suggest_questions.
suggest_questions = "Suggest some data analysis questions that could be answered with this dataset."

# Concatenate the dataset description and the request. Assign to msgs_suggest_questions.
msgs_suggest_questions = [
    SystemMessage(content="You are a data analysis expert."),
    HumanMessage(content=f"{dataset_description}\n\n{suggest_questions}")
]

In [ ]:
chat = ChatOpenAI(openai_api_key=os.environ['OPENAI_KEY'])
rsps_suggest_questions = chat(msgs_suggest_questions)
# Print the response
print("The whole response\n")
print(rsps_suggest_questions)

print("\n----\n")

# Print just the response's content
print("Just the response's content\n")
print(rsps_suggest_questions.content)

print("\n----\n")

# Print the type of the response
print("The type of the response\n")
print(type(rsps_suggest_questions))

### Hold a conversation with GPT

In [ ]:
# Append the response and a new message to the previous messages. 
# Assign to msgs_python_top_models.
msgs_python_top_models = msgs_suggest_questions + [
    rsps_suggest_questions,
    HumanMessage(content="Write some Python code to find the top 5 most popular make/model combinations of electric car in Washington.")
]

# Pass your message to GPT. Assign to rsps_python_top_models.
rsps_python_top_models = chat(msgs_python_top_models)

# Display the response's Markdown content
display(Markdown(rsps_python_top_models.content))

### Execute the code provided by GPT

In [ ]:
# Group the data by make and model, then sum the number of cars registered for each combination
popular_make_model = electric_cars.groupby(['make', 'model'])['n_cars'].sum().reset_index()

# Sort the data by the total number of cars registered in descending order
popular_make_model = popular_make_model.sort_values(by='n_cars', ascending=False)

# Get the top 5 most popular make/model combinations
top_5_popular_make_model = popular_make_model.head(5)

print(top_5_popular_make_model)

In [11]:
# Create a new OpenAI chat object with temperature set to zero. Assign to chat0.
chat0 = ChatOpenAI(temperature=0, openai_api_key=os.environ['OPENAI_KEY'])

In [ ]:
# Ask GPT for code for a bar plot, as detailed in the instructions
msgs_python_plot = msgs_python_top_models + [
    rsps_python_top_models,
    HumanMessage(content="Write some Python code to draw a bar plot of the total count of electric cars by model year, with bars colored by electric vehicle type. Use the Plotly Express package.")
]

rsps_python_plot = chat0(msgs_python_plot)

display(Markdown(rsps_python_plot.content))

In [ ]:
# Call GPT again with the same message list and display the response
rsps_python_plot1 = chat0(msgs_python_plot)

display(Markdown(rsps_python_plot1.content))

In [ ]:
# run the code generated by GPT
import plotly.express as px

# Group the data by model year and electric vehicle type, then sum the number of cars registered for each group
total_count_by_year_type = electric_cars.groupby(['model_year', 'electric_vehicle_type'])['n_cars'].sum().reset_index()

# Plot the bar chart using Plotly Express
fig = px.bar(total_count_by_year_type, x='model_year', y='n_cars', color='electric_vehicle_type',
             labels={'model_year': 'Model Year', 'n_cars': 'Total Count of Electric Cars'},
             title='Total Count of Electric Cars by Model Year',
             barmode='group')

fig.show()